In [1]:
import torch

In [2]:
torch.cuda.is_avaiable()

AttributeError: module 'torch.cuda' has no attribute 'is_avaiable'

In [3]:
torch.cuda.is_available()

True

In [1]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [2]:
#提前确定好在什么设备上训练模型
device='cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [11]:
#定义分类模型网络模块
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),  #第0层 ##线性层即MLP前馈网络，第一个是输入特征的特征维度，
            #第二个是隐含层的大小
            nn.ReLU(),   #非线性激活函数
            nn.Linear(512,512), #第2层
            nn.ReLU(),
            nn.Linear(512,10),#第4层
        )
    def forward(self,x):  #前向运算
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

In [12]:
#用这个模块之前要实例化
model=NeuralNetwork().to(device)
print(model)
#把model搬到了device中

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


用torchsummary中的summary:from torchsummary import summary
把model放入summary中，随便传入一个形状，能看到每一层名称，形状，参数数目等等

In [13]:
#如果要使用model，直接传入数据即可。这一步调用了forward方法，但不需要直接调用
X=torch.rand(1,28,28,device=device)  #在device上初始化
logits=model(X) #作为model的forward函数的一个参数传入进来，最终得到logits，
#作为softmanx层的输入
pred_probab=nn.Softmax(dim=1)(logits) #dim确定对哪一个维度的softmax层进行归一化
#得到的pred_probab是一个预测概率
y_pred=pred_probab.argmax(1) #对得到的概率算出最大值，就能得到每个样本预测的分类值
print(f"Predicted class:{y_pred}")

Predicted class:tensor([3], device='cuda:0')


模型具体的层

In [14]:
input_image=torch.rand(3,28,28) #随机均匀生成一个3*28*28的张量
print(input_image.size())

torch.Size([3, 28, 28])


In [16]:
flatten=nn.Flatten()#nn.Flatten：从第一维到最后一维浓缩成一个维度，只保留batchsize
#和所有其他维度相乘后的维度 即最后保留两个维度
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


28*28=784

In [19]:
layer1=nn.Linear(in_features=28*28,out_features=20) #先将线性层实例化
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
#nn.ReLU:非线性层，非线性激活函数，使神经网络建模能力更强大，通常是在线性变换之后加入到
#网络中  ##类class必须实例化后才可以进行操作
print(f"Before ReLU:{hidden1}\n\n")
hidden1=nn.ReLU()(hidden1)#对ReLU实例化
print(f"After ReLU:{hidden1}")

Before ReLU:tensor([[ 1.0560e-01,  3.3969e-02,  3.7232e-01, -2.3890e-01,  3.5568e-01,
         -2.8359e-01, -4.5223e-01,  3.3610e-01, -7.6581e-03, -3.6852e-01,
          4.0717e-01, -2.8724e-02, -1.6174e-03,  1.8353e-01,  2.9763e-01,
          5.9409e-02, -4.2121e-01, -6.1036e-02,  6.9084e-01, -3.5956e-01],
        [ 3.2734e-01,  8.7424e-02,  3.3259e-01, -6.5141e-02,  5.1348e-01,
         -2.9563e-01, -4.6552e-01,  5.3475e-01,  1.7270e-01, -5.6229e-01,
          5.8176e-01, -2.3416e-01, -3.3694e-02, -2.0387e-01,  7.2927e-01,
          5.1680e-03, -2.0514e-01, -1.8871e-01,  4.9582e-01, -3.0600e-01],
        [ 1.1263e-01,  2.3463e-01,  7.2613e-01,  1.5204e-01,  5.1447e-01,
         -1.0341e-01, -3.8089e-01,  6.4183e-01,  8.6425e-02, -5.2572e-01,
          2.6542e-01, -3.6505e-02,  1.2920e-02, -2.1576e-01,  4.4612e-01,
          5.2473e-04, -3.6033e-01,  2.1169e-01,  5.2817e-01, -2.0145e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU:tensor([[1.0560e-01, 3.3969e-02, 3.7232e-01, 0.000

In [21]:
#nn.Sequential:关于模块有序的容器 （下下次直播具体讲解）
seq_modules=nn.Sequential(#数据会有序地经过以下模块，最终得到一些结果
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

In [22]:
#nn.softmax：指数后的归一化 
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)#关于10个类的分布后的概率分布

In [25]:
#打印模型参数，例如值，大小等
print("Model structure:",model,"\n\n")
for name,param in model.named_parameters():
    print(f"Layer:{name}|Size:{param.size()}|Values:{param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer:linear_relu_stack.0.weight|Size:torch.Size([512, 784])|Values:tensor([[-0.0249, -0.0281, -0.0265,  ..., -0.0224, -0.0049, -0.0328],
        [-0.0141, -0.0014, -0.0018,  ...,  0.0141, -0.0343,  0.0018]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.0.bias|Size:torch.Size([512])|Values:tensor([-0.0033, -0.0157], device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.2.weight|Size:torch.Size([512, 512])|Values:tensor([[ 0.0159, -0.0197,  0.0130,  ..., -0.0361,  0.0075, -0.0092],
        [ 0.0119, -0.0125,  0.0022,  ...,  0.0394, -0.0074, -0.0122]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer:lin